In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
# --- CNN Transforms ---
transform_train_cnn = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_test_cnn = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# --- ViT Transforms ---
transform_train_vit = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
transform_test_vit = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset_cnn = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train_cnn)
test_dataset_cnn = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test_cnn)

train_dataset_vit = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train_vit)
test_dataset_vit = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test_vit)

train_loader_cnn = DataLoader(train_dataset_cnn, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

train_loader_vit = DataLoader(train_dataset_vit, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader_vit = DataLoader(test_dataset_vit, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

classes = train_dataset_cnn.classes

In [ ]:
# CNN: VGG16 pretrained
def get_vgg16_model(num_classes=10, freeze_backbone=True):
    model = models.vgg16(weights="IMAGENET1K_V1")
    model.classifier[6] = nn.Linear(4096, num_classes)

    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.classifier[6].parameters():
            param.requires_grad = True

    return model

# ViT pretrained
from torchvision.models import vit_b_16
def get_vit_model(num_classes=10, freeze_backbone=True):
    model_vit = vit_b_16(weights="IMAGENET1K_V1")
    model_vit.heads.head = nn.Linear(model_vit.heads.head.in_features, num_classes)
    if freeze_backbone:
        for param in model_vit.parameters():
            param.requires_grad = False
        for param in model_vit.heads.head.parameters():
            param.requires_grad = True
    return model_vit

In [ ]:
def run_epoch(model, loader, criterion, optimizer=None, device=None, desc=""):
    is_train = optimizer is not None
    model.train() if is_train else model.eval()

    running_loss = 0.0
    correct, total = 0, 0

    pbar = tqdm(loader, desc=desc, leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        if is_train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(is_train):
            outputs = model(images)
            loss = criterion(outputs, labels)

            if is_train:
                loss.backward()
                optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

        pbar.set_postfix(
            loss=f"{loss.item():.4f}",
            acc=f"{100*correct/total:.2f}%"
        )

    avg_loss = running_loss / len(loader)
    avg_acc = correct / total
    return avg_loss, avg_acc

In [ ]:
def train_model(model, train_loader, val_loader, epochs, lr, device):

    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    if hasattr(model, "heads"):
        optimizer = torch.optim.Adam(model.heads.head.parameters(), lr=lr)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": []
    }

    for epoch in range(epochs):

        train_loss, train_acc = run_epoch(
            model,
            train_loader,
            criterion,
            optimizer,
            device,
            desc=f"Training Epoch {epoch+1}/{epochs}"
        )

        val_loss, val_acc = run_epoch(
            model,
            val_loader,
            criterion,
            optimizer=None,
            device=device,
            desc=f"Validation Epoch {epoch+1}/{epochs}"
        )


        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"TrainLoss: {train_loss:.4f} | TrainAcc: {train_acc*100:.2f}% | "
            f"ValLoss: {val_loss:.4f} | ValAcc: {val_acc*100:.2f}%"

        )

    return history

In [ ]:
def plot_training_curves(results):
    plt.figure(figsize=(14, 8))

    plt.subplot(2,2,1)
    for name, history in results.items():
        plt.plot(history["train_loss"], label=f"{name} Train Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.legend()

    plt.subplot(2,2,2)
    for name, history in results.items():
        plt.plot(history["val_loss"], label=f"{name} Val Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Validation Loss")
    plt.legend()

    plt.subplot(2,2,3)
    for name, history in results.items():
        plt.plot(history["train_acc"], label=f"{name} Train Acc")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Training Accuracy")
    plt.legend()

    plt.subplot(2,2,4)
    for name, history in results.items():
        plt.plot(history["val_acc"], label=f"{name} Val Acc")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Validation Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
results = {}
epochs_list = [3, 5, 7]

print("VGG16 model")
for epoch in epochs_list:
    print(f"\nTraining with epochs={epoch}")
    vgg_model = get_vgg16_model()
    history = train_model(
        vgg_model,
        train_loader_cnn,
        test_loader_cnn,
        epochs=epoch,
        lr=1e-3,
        device=device
    )
    results[f"VGG16_{epoch}ep"] = history

print("VIT model")
for epoch in epochs_list:
    print(f"\nTraining with epochs={epoch}")
    vit_model = get_vit_model()
    history = train_model(
        vit_model,
        train_loader_vit,
        test_loader_vit,
        epochs=epoch,
        lr=3e-4,
        device=device
    )
    results[f"ViT_{epoch}ep"] = history

In [ ]:
plot_training_curves(results)

In [ ]:
results = {}
lrs = [1e-2, 1e-3, 1e-4]

print("VGG16 model")
for lr in lrs:
    print(f"\nTraining with LR={lr}")
    vgg_model = get_vgg16_model()
    history = train_model(
        vgg_model,
        train_loader_cnn,
        test_loader_cnn,
        epochs=5,
        lr=lr,
        device=device
    )
    results[f"VGG16_{lr}lr"] = history

print("VIT model")
for lr in lrs:
    print(f"\nTraining with LR={lr}")
    vit_model = get_vit_model()
    history = train_model(
        vit_model,
        train_loader_vit,
        test_loader_vit,
        epochs=5,
        lr=lr,
        device=device
    )
    results[f"VIT_{lr}lr"] = history

In [ ]:
plot_training_curves(results)

In [ ]:
import random
def show_predictions(model, loader, device, classes, n=6, model_name=""):
    model.eval()

    images, labels = next(iter(loader))
    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

    images = images.cpu()
    labels = labels.cpu()
    predicted = predicted.cpu()

    batch_size = images.size(0)
    n = min(n, batch_size)
    indices = random.sample(range(batch_size), n)

    plt.figure(figsize=(2 * n, 3))

    if model_name:
        plt.suptitle(f"Predictions – {model_name}", fontsize=12)

    for i, idx in enumerate(indices):
        plt.subplot(1, n, i + 1)
        img = images[idx].permute(1, 2, 0)

        img = img * torch.tensor([0.229, 0.224, 0.225]) + \
              torch.tensor([0.485, 0.456, 0.406])
        img = torch.clamp(img, 0, 1)

        plt.imshow(img)
        plt.title(
            f"P: {classes[predicted[idx]]}\nT: {classes[labels[idx]]}",
            fontsize=8
        )
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
show_predictions(
    model=vgg_model,
    loader=test_loader_cnn,
    device=device,
    classes=classes,
    n=6,
    model_name="VGG16"
)

show_predictions(
    model=vit_model,
    loader=test_loader_vit,
    device=device,
    classes=classes,
    n=6,
    model_name="ViT"
)